In [63]:
import numpy as np
import pandas as pd

In [64]:
df = pd.DataFrame([[8,8,4], [7,9,5], [6,10,6], [5,15,7]], columns=['cgpa', 'profile_score', 'lpa'])

In [65]:
df

,cgpa,profile_score,lpa
0,8,8,4
1,7,9,5
2,6,10,6
3,5,15,7


In [66]:
# initializing parameters function for a deep neural network
# layer_dims is a list containing the number of neurons in each layer
def initialize_parameters(layer_dims):

  np.random.seed(3)
  parameters = {}
  L = len(layer_dims)

  for l in range(1, L):

    parameters['W' + str(l)] = np.ones((layer_dims[l - 1], layer_dims[l])) * 0.1
    parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

  return parameters

In [67]:
initialize_parameters([2,2,1]) # testing the function

{'W1': array([[0.1, 0.1],
        [0.1, 0.1]]),
 'b1': array([[0.],
        [0.]]),
 'W2': array([[0.1],
        [0.1]]),
 'b2': array([[0.]])}

In [68]:
# linear forward function for calculating output
def linear_forward(A_prev, W, b):

  Z = np.dot(W.T, A_prev) + b # Z = W.T * A_prev + b

  return Z

In [69]:
# main function for forward propagation
def L_layer_forward(X, parameters):

  A = X
  L = len(parameters) // 2 # number of layers in the neural network

  for l in range(1, L+1):
    A_prev = A # input to the layer
    W1 = parameters['W' + str(l)] # weights
    b1 = parameters['b' + str(l)] # bias
    print("A" + str(l - 1) + ": ", A_prev) # print the input to the layer
    print("W" + str(l) + ": ", W1) # print the weights 
    print("b" + str(l) + ": ", b1) # print the bias
    print("--" * 20) # print 20 spaces

    A = linear_forward(A_prev, W1, b1) # calculate the output of the layer
    print("A" + str(l) + ": ", A) # print the output of the layer
    print("**" * 20)

  return A,A_prev

In [90]:
X = df[['cgpa', 'profile_score']].values[0].reshape(2, 1) # Shape (number of features)
y = df[['lpa']].values[0][0] # Target value

# parameters initialization
parameters = initialize_parameters([2, 2, 1])

# forward propagation
y_hat, A1 = L_layer_forward(X, parameters) # testing the function

A0:  [[8]
 [8]]
W1:  [[0.1 0.1]
 [0.1 0.1]]
b1:  [[0.]
 [0.]]
----------------------------------------
A1:  [[1.6]
 [1.6]]
****************************************
A1:  [[1.6]
 [1.6]]
W2:  [[0.1]
 [0.1]]
b2:  [[0.]]
----------------------------------------
A2:  [[0.32]]
****************************************


In [91]:
y_hat = y_hat[0][0]

In [92]:
A1

array([[1.6],
       [1.6]])

In [93]:
# calculating loss
(y - 0.32)**2 # (y - y^)**2 mse

13.5424

In [94]:
parameters

{'W1': array([[0.1, 0.1],
        [0.1, 0.1]]),
 'b1': array([[0.],
        [0.]]),
 'W2': array([[0.1],
        [0.1]]),
 'b2': array([[0.]])}

In [95]:
# updating weights and bias using GD 
def udpate_parameters(parameters, y, y_hat, A1, X):
    
  parameters['W2'][0][0] = parameters['W2'][0][0] + (0.001 * 2 * (y - y_hat) * [0][0]) # updating weights
  parameters['W2'][1][0] = parameters['W2'][1][0] + (0.001 * 2 * (y - y_hat) * [1][0])
  parameters['b2'][0][0] = parameters['W2'][1][0] + (0.001 * 2 * (y - y_hat))

  parameters['W1'][0][0] = parameters['W1'][0][0] + (0.001 * 2 * (y - y_hat) * parameters['W2'][0][0] * X[0][0])
  parameters['W1'][0][1] = parameters['W1'][0][1] + (0.001 * 2 * (y - y_hat) * parameters['W2'][0][0] * X[1][0])
  parameters['b1'][0][0] = parameters['b1'][0][0] + (0.001 * 2 * (y - y_hat) * parameters['W2'][0][0])

  parameters['W1'][1][0] = parameters['W1'][1][0] + (0.001 * 2 * (y - y_hat) * parameters['W2'][1][0] * X[0][0])
  parameters['W1'][1][1] = parameters['W1'][1][1] + (0.001 * 2 * (y - y_hat) * parameters['W2'][1][0] * X[1][0])
  parameters['b1'][1][0] = parameters['b1'][1][0] + (0.001 * 2 * (y - y_hat) * parameters['W2'][1][0])


In [96]:
X = df[['cgpa', 'profile_score']].values[0].reshape(2, 1) # Shape (number of features)
y = df[['lpa']].values[0][0] # Target value

# parameters initialization
parameters = initialize_parameters([2, 2, 1])

y_hat, A1 = L_layer_forward(X, parameters) # testing the function
y_hat = y_hat[0][0] # get the predicted value

# updating weights and bias using GD
udpate_parameters(parameters, y, y_hat, A1, X)

parameters

A0:  [[8]
 [8]]
W1:  [[0.1 0.1]
 [0.1 0.1]]
b1:  [[0.]
 [0.]]
----------------------------------------
A1:  [[1.6]
 [1.6]]
****************************************
A1:  [[1.6]
 [1.6]]
W2:  [[0.1]
 [0.1]]
b2:  [[0.]]
----------------------------------------
A2:  [[0.32]]
****************************************


{'W1': array([[0.105888  , 0.105888  ],
        [0.10632136, 0.10632136]]),
 'b1': array([[0.000736  ],
        [0.00079017]]),
 'W2': array([[0.1    ],
        [0.10736]]),
 'b2': array([[0.11472]])}

In [98]:
# epochs implementation

parameters = initialize_parameters([2, 2, 1])
epochs = 75

for i in range(epochs):
  loss = [] # loss per epoch

  for j in range(df.shape[0]):

    X = df[['cgpa', 'profile_score']].values[j].reshape(2, 1) # Shape (number of features)
    y = df[['lpa']].values[j][0] # Target value

    # parameters initialization

    y_hat, A1 = L_layer_forward(X, parameters) # testing the function
    y_hat = y_hat[0][0] # get the predicted value

    # updating weights and bias using GD
    udpate_parameters(parameters, y, y_hat, A1, X)

    loss.append((y - y_hat)**2) # calculate loss

  print(f"Epoch {i+1}/{epochs} Loss: {sum(loss)/len(loss)}")

parameters

A0:  [[8]
 [8]]
W1:  [[0.1 0.1]
 [0.1 0.1]]
b1:  [[0.]
 [0.]]
----------------------------------------
A1:  [[1.6]
 [1.6]]
****************************************
A1:  [[1.6]
 [1.6]]
W2:  [[0.1]
 [0.1]]
b2:  [[0.]]
----------------------------------------
A2:  [[0.32]]
****************************************
A0:  [[7]
 [9]]
W1:  [[0.105888   0.105888  ]
 [0.10632136 0.10632136]]
b1:  [[0.000736  ]
 [0.00079017]]
----------------------------------------
A1:  [[1.69884421]
 [1.69889838]]
****************************************
A1:  [[1.69884421]
 [1.69889838]]
W2:  [[0.1    ]
 [0.10736]]
b2:  [[0.11472]]
----------------------------------------
A2:  [[0.46699815]]
****************************************
A0:  [[ 6]
 [10]]
W1:  [[0.1122342  0.1140474 ]
 [0.11370999 0.11582102]]
b1:  [[0.0016426 ]
 [0.00184569]]
----------------------------------------
A1:  [[1.81214768]
 [1.84434035]]
****************************************
A1:  [[1.81214768]
 [1.84434035]]
W2:  [[0.1     ]
 [0.116426

{'W1': array([[0.2453059 , 0.42275277],
        [0.41412649, 0.91151119]]),
 'b1': array([[0.02696261],
        [0.06445399]]),
 'W2': array([[0.1       ],
        [0.36962607]]),
 'b2': array([[0.36967839]])}